In [1]:
!curl https://download.mosek.com/stable/8.1.0.82/mosektoolslinux64x86.tar.bz2 --output /root/mosektoolslinux64x86.tar.bz2
!cd /root/; tar xvf /root/mosektoolslinux64x86.tar.bz2
from google.colab import drive
drive.mount('/content/gdrive')
!cp -avr /content/gdrive/My\ Drive/Mosek/mosek.lic /root/mosek/
!pip install git+https://github.com/convexengineering/gpkit
!pip show gpkit

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 40.3M  100 40.3M    0     0   247M      0 --:--:-- --:--:-- --:--:--  247M
mosek/8/tools/examples/python/parameters.py
mosek/8/toolbox/examples/cqo1.m
mosek/8/toolbox/r2014a/mskqpopt.m
mosek/8/tools/examples/fusion/java/breaksolver.java
mosek/8/toolbox/r2014a/mskcheck.m
mosek/8/tools/examples/fusion/cxx/lo1.cc
mosek/8/tools/examples/data/dgo.f
mosek/8/tools/examples/dotnet/scopt1.cs
mosek/8/tools/examples/dotnet/solvebasis.cs
mosek/8/toolbox/r2014aom/mskerrmsg.m
mosek/8/toolbox/examples/simple.m
mosek/8/tools/examples/java/solutionquality.java
mosek/8/tools/examples/fusion/dotnet/production.cs
mosek/8/tools/examples/fusion/java/Makefile
mosek/8/tools/examples/fusion/python/lo1.py
mosek/8/tools/examples/fusion/java/duality.java
mosek/8/tools/platform/linux64x86/bin/libcilkrts.so.5
mosek/8/toolbox/examples/qo2.m
mosek/8/tools/ex

In [2]:
from collections import deque
import os
import networkx as nx
import re
import glob
import gpkit as gp
import sys
import tqdm
#import matplotlib.pyplot as plt

sys.setrecursionlimit(10000)

.

Found no installed solvers, beginning a build.
# Building GPkit version 1.1
# Moving to the directory from which GPkit was imported.

Attempting to find and build solvers:

# Looking for `mosek_cli`
#   (A "success" is if mskexpopt complains that
#    we haven't specified a file for it to open.)
#     Calling 'mskexpopt'
##
### CALL BEGINS
### CALL ENDS
##
# Looks like `mskexpopt` was not found in the default PATH,
#  so let's try locating that binary ourselves.
#   Adding /root/mosek/8/tools/platform/linux64x86/bin to the PATH
#     Calling 'mskexpopt'
##
### CALL BEGINS
### CALL ENDS
##

Found mosek_cli in /root/mosek/8/tools/platform/linux64x86/bin

# Looking for `mosek_conif`
#   Trying to import mosek...
# Did not find
# mosek_conif

# Looking for `cvxopt`
#   Trying to import cvxopt...

Found cvxopt in the default PYTHONPATH

Found the following solvers: mosek_cli, cvxopt
#     Replacing directory env


...........................................................F....................................................................................................................................................
FAIL: test_numerical_precision (gpkit.tests.t_nomials.TestMonomial)
not sure what to test here, placeholder for now
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gpkit/tests/t_nomials.py", line 195, in test_numerical_precision
    self.assertEqual(np.log((m1**4 * m2**3).c),  # pylint: disable=no-member
AssertionError: 43.49427395780073 != 43.494273957800715

----------------------------------------------------------------------
Ran 209 tests in 6.393s

FAILED (failures=1)



GPkit is now installed with solver(s) ['mosek_cli', 'cvxopt']
To incorporate new solvers at a later date, run `gpkit.build()`.

If any tests didn't pass, please post the output above
(starting from "Found no installed solvers, beginning a build.")
to gpkit@mit.edu or https://github.com/convexengineering/gpkit/issues/new
so we can prevent others from having these errors.

The same goes for any other bugs you encounter with GPkit:
send 'em our way, along with any interesting models, speculative features,
comments, discussions, or clarifications you feel like sharing.

Finally, we hope you find our documentation (https://gpkit.readthedocs.io/)
and engineering-design models (https://github.com/convexengineering/gplibrary/)
to be useful resources for your own applications.

Enjoy!



In [3]:
g = {'NOT':lambda n : 1, 'NAND': lambda n : (int(n) + 2)/3, 'NOR': lambda n : (2*int(n) + 1)/3}                   #logical effort
p = lambda n: int(n)                                                                                              #parasitic effort

In [4]:
class Netlist:
    def __init__(self,file=None,parse=True):
        self.file=file
        self.DAG=nx.DiGraph()
        if parse:
          self.parse()
          
    def __repr__(self):
        return f"{self.DAG}"
    
    def gates_adjacency(self):
        adj_list=dict()
        for node in self.DAG:
            if 'type' in self.DAG.nodes[node].keys():
                adj_list[gate:=node]=[]
                for net in self.DAG.successors(gate):
                  adj_list[gate].extend(self.DAG.successors(net))
        return adj_list

    def longest_path(self):
        ''' Find the path with the largest depth and 
        skip every other node starting from second 
        node as they are not gates '''
        path = nx.dag_longest_path(self.DAG)[1::2]
        #path = [path[idx] for idx in range(1,len(path),2)]
        path = [f"{self.DAG.nodes[node]['type']}_{node}" for node in path]
        #print(f"depth = {len(path)}")
        #print(" ==> ".join(path))
        return "=>".join(path)

    def parse(self):
        parser_re = re.compile("""([A-Z]+\d)_  #matches the gate type
                                  (\d+)        #matches the gate index
                                  \s
                                  \(([^\)]*)\) #matches input and output nets of a gate""",
                               re.VERBOSE)  
        with open(self.file,'r') as f:
            text=f.read()

        for match in re.finditer(parser_re,text):
            gate_type, gate_index = match.group(1, 2)
            out, *ins = [net.strip() for net in match.group(3).split(',')]
            for net in ins:
                self.DAG.add_edge(net, gate_index)
            self.DAG.add_edge(gate_index, out)
            self.DAG.nodes[gate_index]['type'] = gate_type
            self.DAG.nodes[gate_index]['size']=gp.Variable(f"x_{gate_index}")  

In [13]:
class GPproblem:
  min_size=1            #minimum allowable size of each gate
  max_size=100          #maximum allowable size of each gate
  C_L=1000              #load capacitance at all the outputs
  C_max=50              #maximum capacitance at all the inputs
  #logical effort
  g={"NOT": lambda n: 1 , "NAND":lambda n: (int(n)+2)/3, 
     "NOR": lambda n: (2*int(n)+1)/3} 

  def __init__(self,netlist,mode="Timing",model=False,T_spec=None):
    self.netlist=netlist
    self.mode=mode
    self.T_spec=T_spec
    if model:
      self.model()

  def __repr__(self):
    return f"GPproblem for {self.netlist}" 

  def model(self):
    constraints=[]
    DAG=self.netlist.DAG
    gates_adjacency=self.netlist.gates_adjacency()
    nodes_list=DAG.nodes
    T_spec=gp.Variable("T_wall")
    objective=T_spec
    tol=0.000001 # input arrival time tolerance
    for gate in gates_adjacency:
      constraints.append(nodes_list[gate]["size"]>=self.min_size)
      constraints.append(nodes_list[gate]["size"]<=self.max_size)

    if self.mode=="Area":
      objective=0
      T_spec=self.T_spec
      for gate in gates_adjacency:
        objective += nodes_list[gate]["size"]

    for node in nx.topological_sort(DAG):
      if not DAG.in_degree(PI:=node):
        nodes_list[PI]["arrival_time"]=gp.Variable(f"a_{PI}")
        constraints.append(nodes_list[PI]["arrival_time"] == tol)
        for gate in DAG.successors(PI):
          gate_type=nodes_list[gate]["type"][:-1]
          gate_size=nodes_list[gate]["size"]
          n_inputs=nodes_list[gate]["type"][-1]
          constraints.append(self.g[gate_type](n_inputs)*gate_size <= 
                             self.C_max)

      elif (gate:=node) in gates_adjacency:
        fanout_net=list(DAG.successors(gate))[0]
        gate_type=nodes_list[gate]["type"][:-1]
        n_inputs= nodes_list[gate]["type"][-1]
        gate_size=nodes_list[gate]["size"]
        nodes_list[fanout_net]["arrival_time"]=gp.Variable(f"a_{fanout_net}")
        parasitic_effort = int(n_inputs)
        stage_effort=0
        for fanout_gate in DAG.successors(fanout_net):
          fanout_gate_size=nodes_list[fanout_gate]["size"]
          fanout_gate_type=nodes_list[fanout_gate]["type"][:-1]
          n_inputs=nodes_list[fanout_gate]["type"][-1]
          stage_effort+=fanout_gate_size*self.g[fanout_gate_type](n_inputs)
        if not stage_effort:
          stage_effort+= self.C_L
          constraints.append(nodes_list[PO:=fanout_net]["arrival_time"]<=
                             T_spec)
        stage_effort /= gate_size
        d=stage_effort + parasitic_effort

        for fanin_net in DAG.predecessors(gate):
          constraints.append(d+nodes_list[fanin_net]["arrival_time"]<=
                             nodes_list[fanout_net]["arrival_time"])
      else:
        continue
    return  objective,constraints   

In [42]:
netlist = Netlist('c880.txt')
print(netlist.gates_adjacency())
print(netlist.longest_path())
obj,constraints = GPproblem(netlist,"Timing").model()
#obj,constraints = GPproblem(netlist,"Area",T_spec=171).model()
model = gp.Model(obj,constraints)
sol = model.solve(verbosity=1)
print(sol['variables']['T_wall'])


{}



UnboundedGP: ignored

In [ ]:
file="./midsem.v"
netlist=Netlist(file)
DAG=netlist.DAG
for node in DAG.nodes:
  print(list(DAG.successors(node)))
  break
objective,constraints=GPproblem(netlist).model()
constraints[1]= (gp.Variable(f"x_{1}")==1)
constraints=constraints[1:]
for constraint in constraints:
  print(constraint)

['1']
x_1 = 1
x_2 ≥ 1
x_2 ≤ 100
x_3 ≥ 1
x_3 ≤ 100
x_4 ≥ 1
x_4 ≤ 100
a_A = 1e-06
a_N1 = 1e-06
a_N2 = 1e-06
a_N4 = 1e-06
a_N6 = 1e-06
a_N0 ≥ 2.33·x_2/x_1 + 1 + a_A
a_N3 ≥ 1.33·x_3/x_2 + 3 + a_N0
a_N3 ≥ 1.33·x_3/x_2 + 3 + a_N1
a_N3 ≥ 1.33·x_3/x_2 + 3 + a_N2
a_N5 ≥ 1.67·x_4/x_3 + 2 + a_N3
a_N5 ≥ 1.67·x_4/x_3 + 2 + a_N4
a_Y ≤ T_wall
a_Y ≥ 15.6/x_4 + 2 + a_N5
a_Y ≥ 15.6/x_4 + 2 + a_N6


In [ ]:
model=gp.Model(objective,constraints)

In [ ]:
model.solve(verbosity=0)